In [1]:
from confluent_kafka.admin import AdminClient, NewTopic

bootstrap_servers = 'localhost:9092'  
admin_client = AdminClient({'bootstrap.servers': bootstrap_servers})

topic_name = 'mobilPriceTopic'  
num_partitions = 5  
replication_factor = 1  

new_topic = NewTopic(topic=topic_name, num_partitions=num_partitions, replication_factor=replication_factor)
admin_client.create_topics([new_topic])


{'mobilPriceTopic': <Future at 0x24f4f11cdd0 state=running>}

In [16]:
from confluent_kafka.admin import AdminClient

bootstrap_servers = 'localhost:9092'
admin_client = AdminClient({'bootstrap.servers': bootstrap_servers})

topics_metadata = admin_client.list_topics().topics
print("Mevcut Topicler:")
for topic in topics_metadata:
    print(topic)

Mevcut Topicler:
mobil_testdata_topic
mobilPriceTopic
jsonTopic
__consumer_offsets


In [17]:
#test.csv dosyasi topice ekleme
from confluent_kafka import Producer
import csv

bootstrap_servers = 'localhost:9092'

producer = Producer({'bootstrap.servers': bootstrap_servers})

csv_file = 'test.csv'

with open(csv_file, newline='') as file:
    reader = csv.reader(file)
    next(reader)  
    for row in reader:
        producer.produce('mobilPriceTopic', value=','.join(row))

producer.flush()


0

In [15]:
#topice eklenen verileri test
from confluent_kafka import Consumer, KafkaException
import sys

# Kafka broker'ının adresi ve portu
bootstrap_servers = 'localhost:9092'

# Kafka konusu
topic = 'mobilPriceTopic'

# Consumer oluşturma
conf = {'bootstrap.servers': bootstrap_servers, 'group.id': 'mygroup', 'auto.offset.reset': 'earliest'}
consumer = Consumer(conf)

# Kafka konusuna abone olma
consumer.subscribe([topic])

try:
    while True:
        # Mesajları tüketme
        msg = consumer.poll(timeout=1.0)

        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaException._PARTITION_EOF:
                continue
            else:
                print(msg.error())
                break

        # Mesajları ekrana yazdırma
        print('Received message: {}'.format(msg.value().decode('utf-8')))

except KeyboardInterrupt:
    sys.exit(0)

finally:
    # Consumer'ı kapatma
    consumer.close()


AssertionError: 